## Jaccard Similarity
Jaccard similarity is defined between two sets as the ratio of the volume of their intersection divided by the volume of their union. In the context of graphs, the neighborhood of a vertex is seen as a set. The Jaccard similarity weight of each edge represents the strength of connection between vertices based on the relative similarity of their neighbors. For further detail see [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index)

To Compute the Jaccard similarity between each pair of vertices connected by an edge in cuGraph use: 
**nvJaccard(G)**
* G: A cugraph.Graph object

Returns:
jaccard_weights: cudf.DataFrame with three columns:
* jaccard_weights['source']: The source vertex id.
* jaccard_weights['destination']: The destination vertex id.
* jaccard_weights['jaccard_coeff']: The jaccard coefficient computed between the source and destination vertex.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

import numpy as np


In [2]:
# define a function for printing out the top most similar vertices
def printMostSimilar(df):
    bestEdge = 0
    
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
            bestEdge = i
        
    print("Vertices " + str(df['source'][bestEdge]) + " and " + 
          str(df['destination'][bestEdge]) + " are most similar with score: " 
          + str(df['jaccard_coeff'][bestEdge]))

In [15]:
# define a function for printing out the top most similar vertices
def print_gt_threshold(df, limit):
    
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > limit:
            print("Vertices " + str(df['source'][i]) + " and " + 
                  str(df['destination'][i]) + " are most similar with score: " + 
                  str(df['jaccard_coeff'][i]))

### Reading a CSV file using cuDF

In [16]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [17]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [18]:
# Let's look at the DataFrame. There should be two columns and 154 records
gdf

<cudf.DataFrame ncols=2 nrows=154 >

In [19]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

,src,dst
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


### Ceate a Graph and call the Jaccard analytic

In [20]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [21]:
# How many vertices are in the graph?  Remember that Graph is zero based while teh data start at vertex 1
G.num_vertices()

35

In [22]:
# Call cugraph.nvJaccard 
df = cugraph.nvJaccard(G)

In [23]:
printMostSimilar(df)

Vertices 33 and 34 are most similar with score: 0.5555556


In [26]:
print_gt_threshold(df, 0.4)

Vertices 4 and 8 are most similar with score: 0.42857143
Vertices 8 and 4 are most similar with score: 0.42857143
Vertices 33 and 34 are most similar with score: 0.5555556


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___